## Sampling Errors and other Hardware Corrections

Sitting somewhere between I/O and preprocessing, the methods in the notebook are intended to correct flaws in the data caused on the acquisition hardware side.

### Nonpositive or NaN values in amplitude

Sometimes, in noisy channels background light subtraction can lead to nonpositive amplitude values. Also, single samples of data can go missing and show as NaN values. We can simulate this by loading an example dataset and altering the data.

In [ ]:
# This cells setups the environment when executed in Google Colab.
try:
    import google.colab
    !curl -s https://raw.githubusercontent.com/ibs-lab/cedalion/colab_setup/scripts/colab_setup.py -o colab_setup.py
    # Select branch with --branch "branch name" (default is "dev")
    %run colab_setup.py
except ImportError:
    pass

In [ ]:
import matplotlib.pyplot as p
import numpy as np
import xarray as xr

import cedalion
import cedalion.dataclasses as cdc
import cedalion.datasets
import cedalion.sigproc.quality as quality
import pint_xarray

%matplotlib widget

# Load the dataset
rec = cedalion.datasets.get_fingertappingDOT()

display(rec["amp"])

# Set values to simulate hardware issues
unit = rec["amp"].pint.units
chwl = dict(channel="S1D1", wavelength="850")
rec["amp"].sel(**chwl).isel(time=slice(8,10))[:] = np.nan * unit
rec["amp"].sel(**chwl).isel(time=slice(19,20))[:] = 0 * unit

# Plot first 30 sec
f, ax = p.subplots(1, 1, figsize=(12, 4))
ax.plot(rec["amp"].time.isel(time=slice(0,30)), rec["amp"].sel(**chwl).isel(time=slice(0,30)))
ax.set_ylim(0.138, 0.148)
ax.set_title("Amplitude")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Amplitude (a.u.)")
p.show()

The repair_amp function handles nonpositive amp values and optionally fills NaNs using the xarray interpolate_na method. It replaces nonpositive values with a small magnitude and runs a rolling median filter over the data. This method works best for isolated nonpositive values.

In [ ]:
# Correct sample issues from hardware
rec["amp_proc"] = quality.repair_amp(rec["amp"], median_len=3, interp_nan=True)


# Plot first 30 sec
f, ax = p.subplots(1, 1, figsize=(12, 4))
ax.plot(rec["amp_proc"].time.isel(time=slice(0,30)), rec["amp_proc"].sel(**chwl).isel(time=slice(0,30)))
ax.set_title("Amplitude")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Amplitude (a.u.)")
ax.set_ylim(0.138, 0.148)
p.show()